In [1]:
import numpy as np
import pandas as pd
import os
import sqlalchemy as sa

from dotenv import load_dotenv

import constants as consts

In [2]:
df = pd.read_csv(
    '../../data/nyc_housing_rec/raw_data/311_calls.csv',
    usecols = consts.keep_cols,
)

/Users/ericphillips/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2368133 entries, 0 to 2368132
Data columns (total 8 columns):
 #   Column          Dtype  
---  ------          -----  
 0   Unique Key      int64  
 1   Created Date    object 
 2   Agency          object 
 3   Complaint Type  object 
 4   Location Type   object 
 5   Incident Zip    object 
 6   Latitude        float64
 7   Longitude       float64
dtypes: float64(2), int64(1), object(5)
memory usage: 144.5+ MB


In [4]:
# Check the NA vaules for each column

df.isna().sum()

Unique Key             0
Created Date           0
Agency                 0
Complaint Type         0
Location Type     390379
Incident Zip       67059
Latitude           64957
Longitude          64957
dtype: int64

In [5]:
# Convert the Incident Zip column to Int32

def to_ints(series):
  a = pd.to_numeric(series, errors = 'coerce')

  try:
    return a.astype('Int32')

  except ValueError:
    return a

df['Incident Zip'] = to_ints(df['Incident Zip'])

In [6]:
df['Incident Zip'].isna().value_counts()

False    2301073
True       67060
Name: Incident Zip, dtype: int64

In [7]:
df.iloc[0]

Unique Key                          48544884
Created Date          01/01/2021 12:00:00 AM
Agency                                 DOHMH
Complaint Type                Food Poisoning
Location Type     Restaurant/Bar/Deli/Bakery
Incident Zip                           11209
Latitude                           40.619073
Longitude                         -74.032865
Name: 0, dtype: object

In [8]:
# This is how we will index the time

df.loc[0, 'Created Date'][11:]

'12:00:00 AM'

In [9]:
# This is how we will index the date

df.iloc[0]['Created Date'][:10]

'01/01/2021'

In [10]:
# Creates a new column of the time

df['Created Time'] = df['Created Date'].str[11:]

In [11]:
# Removes the time information from the Created Date

df['Created Date'] = df['Created Date'].str[:10]

In [12]:
# Replace all NaN values with None.
# This will help with created the table with sqlalchemy

df = df.replace({np.nan: None})

In [13]:
reorder = [
 'Unique Key',
 'Created Date',
 'Created Time',   
 'Agency',
 'Complaint Type',
 'Location Type',
 'Incident Zip',
 'Latitude',
 'Longitude',
]

df = df[reorder]

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2368133 entries, 0 to 2368132
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   Unique Key      int64 
 1   Created Date    object
 2   Created Time    object
 3   Agency          object
 4   Complaint Type  object
 5   Location Type   object
 6   Incident Zip    object
 7   Latitude        object
 8   Longitude       object
dtypes: int64(1), object(8)
memory usage: 162.6+ MB


In [15]:
# Rename the columns for easier querying

df.columns = consts.new_col_values

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2368133 entries, 0 to 2368132
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   unique_key      int64 
 1   created_date    object
 2   created_time    object
 3   agency          object
 4   complaint_type  object
 5   location_type   object
 6   incident_zip    object
 7   latitude        object
 8   longitude       object
dtypes: int64(1), object(8)
memory usage: 162.6+ MB


In [18]:
# Will need to assign primary key in PGAdmin.

load_dotenv()

engine = sa.create_engine(os.getenv('ENGINE'))

df.to_sql(
    'calls_311',
    engine,
    if_exists = 'replace', # This will drop the table if it already exists. Delete this line if necessary.
    index = False,
    dtype = {
        'unique_key': sa.types.INTEGER(),
        'created_date': sa.types.Date(),
        'created_time': sa.types.Time(),
        'agency': sa.types.Text(),
        'complaint_type': sa.types.Text(),
        'incident_zip': sa.types.INTEGER(),
        'latitude': sa.types.FLOAT(),
        'longitude': sa.types.FLOAT()
    }
)